## Interact with NGSI-LD context broker usign the data model

Uses orion-ld broker

In [105]:
from ngsildclient import Entity, Client

In [106]:
#client = Client(hostname="localhost", port=8080)
client = Client(hostname="localhost", port=1026, port_temporal=8080)

Connected to Context Broker at localhost:1026 | tenant=N/A | vendor=Orion-LD | version=1.1.2

In [28]:
%%time
ctx = "https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/93bff748cfdba3a557eca1952e0d3705f83b243e/context.jsonld"
for idx in range(1000):
    animal = Entity("Animal", f"cow{idx}", ctx=ctx)
    animal.prop("breed", "AY")
    animal.prop("legalid", idx)
    client.create(animal)

CPU times: user 1.68 s, sys: 98.1 ms, total: 1.78 s
Wall time: 5.51 s


In [29]:
client.delete(client.query("Animal", ctx=ctx))

Entities deleted : 1000/1000 [1.00]

op: delete, success: 1000, errors: 0

In [30]:
%%time
cows = []
for idx in range(1000):
    animal = Entity("Animal", f"cow{idx}", ctx=ctx)
    animal.prop("breed", "AY")
    animal.prop("legalid", idx)
    cows.append(animal)
client.create(cows)   

Entities created : 1000/1000 [1.00]

CPU times: user 69.1 ms, sys: 0 ns, total: 69.1 ms
Wall time: 941 ms


op: create, success: 1000, errors: 0

In [31]:
%%timeit
cows = client.query("Animal", ctx=ctx)

94.6 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
#Clean up
#client.delete(client.query("Animal", ctx=ctx))

Entities deleted : 1000/1000 [1.00]

op: delete, success: 1000, errors: 0

### Check entity types

In [33]:
import requests
import json
es = requests.get("http://localhost:1026/ngsi-ld/v1/types").json()
print(json.dumps(es, indent=2))

{
  "id": "urn:ngsi-ld:EntityTypeList:3713a536-ce2b-11ed-91e5-0242ac120107",
  "type": "EntityTypeList",
  "typeList": [
    "https://smartdatamodels.org/dataModel.Agrifood/Animal",
    "https://uri.fiware.org/ns/dataModels#Building",
    "https://uri.fiware.org/ns/dataModels#Person"
  ]
}


## Time series data

In [71]:
# Testing with
# https://ngsi-ld-tutorials.readthedocs.io/en/latest/short-term-history.html
import pandas as pd
ctx = "http://context/ngsi-context.jsonld"
tclient = Client(port=1026, port_temporal=8080, tenant="openiot")

Connected to Context Broker at localhost:1026 | tenant=openiot | vendor=Orion-LD | version=1.1.2

In [130]:
animals = tclient.query(type="Animal", ctx=ctx)
%time a1 = tclient.temporal.get(animals[1])
print(len(a1))
#pd.DataFrame(a1["https://purl.bioontology.org/ontology/MESH/D006339"]["values"])

CPU times: user 2.27 ms, sys: 0 ns, total: 2.27 ms
Wall time: 303 ms
5


In [124]:
import datetime
datetime.datetime.now().isoformat()

'2023-03-29T19:17:24.205281'

In [120]:
import requests

url = 'http://localhost:8080/temporal/entities/'
headers = {
    'NGSILD-Tenant': 'openiot',
    'Link': '<http://context/ngsi-context.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': 'Animal',
    'pageSize': 100,
    'lastN': 3,
    'q': 'sex=="male"',
    'timeproperty': 'modifiedAt',
    'options': 'count',
    'attrs': 'sex,heartRate',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}',
}

response = requests.get(url, headers=headers, params=params)

response.json()

[{'id': 'urn:ngsi-ld:Animal:pig021',
  'type': 'Animal',
  'sex': {'type': 'Property',
   'value': 'male',
   'modifiedAt': '2023-03-29T11:45:37.453Z',
   'instanceId': 'urn:ngsi-ld:attribute:instance:39dae580-ce27-11ed-b3fe-0242ac120107'}},
 {'id': 'urn:ngsi-ld:Animal:pig011',
  'type': 'Animal',
  'sex': {'type': 'Property',
   'value': 'male',
   'modifiedAt': '2023-03-29T11:45:37.453Z',
   'instanceId': 'urn:ngsi-ld:attribute:instance:39dab290-ce27-11ed-b3fe-0242ac120107'}},
 {'id': 'urn:ngsi-ld:Animal:pig025',
  'type': 'Animal',
  'sex': {'type': 'Property',
   'value': 'male',
   'modifiedAt': '2023-03-29T11:45:37.453Z',
   'instanceId': 'urn:ngsi-ld:attribute:instance:39daf994-ce27-11ed-b3fe-0242ac120107'}},
 {'id': 'urn:ngsi-ld:Animal:pig003',
  'type': 'Animal',
  'sex': {'type': 'Property',
   'value': 'male',
   'modifiedAt': '2023-03-29T11:45:37.453Z',
   'instanceId': 'urn:ngsi-ld:attribute:instance:39da8c66-ce27-11ed-b3fe-0242ac120107'}},
 {'id': 'urn:ngsi-ld:Animal:pig0

In [121]:
%%time 
response = requests.get(url, headers=headers, params=params)
print(len(response.json()))

17
CPU times: user 4.54 ms, sys: 437 µs, total: 4.98 ms
Wall time: 2.71 s
